In [2]:
!source /opt/intel/oneapi/setvars.sh

 
:: initializing oneAPI environment ...
   bash: BASH_VERSION = 5.1.16(1)-release
   args: Using "$@" for setvars.sh arguments: 
:: ccl -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: ipp -- latest
:: ippcp -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: oneAPI environment initialized ::
 


In [1]:
import pandas as pd
import procesing_pdf as ppdf
from embedding_model import extract_features
import re
import os
import numpy as np


/home/renato/miniforge3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/renato/miniforge3/envs/llm/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: ''If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/renato/miniforge3/envs/llm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/renato/miniforge3

512


In [2]:
df1=pd.read_excel("train_base.xlsx",index_col=False)
df2=pd.read_excel("train_base_2.xlsx",index_col=False)

FileNotFoundError: [Errno 2] No such file or directory: 'train_base_2.xlsx'

In [39]:
columns_to_concatenate = [7] + list(range(19, 31))

# Extraer las columnas de ambos DataFrames
df1_selected = df1.iloc[:, columns_to_concatenate]
df2_selected = df2.iloc[:, columns_to_concatenate]

# Concatenar las filas de df2 debajo de df1
result = pd.concat([df1_selected, df2_selected], ignore_index=True)
result

,Title,"Business, Firms, and Finance","Conflict, Peace, and Security",Education and Human Development,Climate and Energy,Gender and Inclusion,Health and Wellbeing,"Culture, Institutions, and History",Labor and Urban Economics,"Governance, Political Economy, and Public Management",Social Welfare and Public Finance,Technology and Data Science,"Trade, Growth, and Regional Economics"
0,Facilitating learning and discovery-oriented i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes
1,Responding Adaptively to the COVID-19 Crisis i...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,Yes,NaN,NaN,NaN
2,Vaccines for a sustainable planet,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
3,Estimates and Projections of the Global Econom...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
4,Cohort Profile: Real-Time Insights of COVID-19...,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,"Aspirations, Social Norms and Development",NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN
241,"Entertainment, Education and Attitudes towards...",NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN
242,How ed-tech can help leapfrog progress in educ...,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
243,Behavioral Inattention,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
result.to_excel("train_base_concatenated.xlsx", index=False)

NameError: name 'result' is not defined

In [4]:
def replace_colons_with_underscores(text):
    invalid_characters = r'[<>:"/\\|?*]'
    return re.sub(invalid_characters, '_', text)
download_dir="/home/renato/Proyects/Work/scraping/pdf_train"

In [6]:
mapping = {
    'Yes': 1,
    np.nan: 0
}
df=pd.read_excel("train_base_concatenated.xlsx",index_col=False)
cols_to_replace = df.columns[-12:]
df[cols_to_replace] = df[cols_to_replace].applymap(lambda x: mapping.get(x, 0))
df

C:\Users\renat\AppData\Local\Temp\ipykernel_17800\334298763.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[cols_to_replace] = df[cols_to_replace].applymap(lambda x: mapping.get(x, 0))


,Title,"Business, Firms, and Finance","Conflict, Peace, and Security",Education and Human Development,Climate and Energy,Gender and Inclusion,Health and Wellbeing,"Culture, Institutions, and History",Labor and Urban Economics,"Governance, Political Economy, and Public Management",Social Welfare and Public Finance,Technology and Data Science,"Trade, Growth, and Regional Economics"
0,Facilitating learning and discovery-oriented i...,0,0,0,0,0,0,0,0,1,0,0,1
1,Responding Adaptively to the COVID-19 Crisis i...,0,0,0,0,0,1,0,0,1,0,0,0
2,Vaccines for a sustainable planet,0,0,0,1,0,1,0,0,0,0,0,0
3,Estimates and Projections of the Global Econom...,0,0,0,0,0,1,0,0,0,0,0,0
4,Cohort Profile: Real-Time Insights of COVID-19...,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,"Aspirations, Social Norms and Development",0,0,1,0,0,0,1,0,1,0,0,0
241,"Entertainment, Education and Attitudes towards...",0,0,0,0,1,0,1,0,0,0,0,0
242,How ed-tech can help leapfrog progress in educ...,0,0,1,0,0,0,0,0,0,0,1,0
243,Behavioral Inattention,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
rows_to_keep = []
for index, row in df.iterrows():
    pdf_filename = f"{row['Title']}.pdf"
    pdf_filename=replace_colons_with_underscores(pdf_filename)
    pdf_path = os.path.join(download_dir, pdf_filename)
    if os.path.exists(pdf_path):
        text=ppdf.get_clean_text(replace_colons_with_underscores(row['Title']))
        embedingtext=extract_features(text)
        df.at[index, 'Title'] = embedingtext
        rows_to_keep.append(index)
    else:
        pass
df = df.loc[rows_to_keep]
df


,Title,"Business, Firms, and Finance","Conflict, Peace, and Security",Education and Human Development,Climate and Energy,Gender and Inclusion,Health and Wellbeing,"Culture, Institutions, and History",Labor and Urban Economics,"Governance, Political Economy, and Public Management",Social Welfare and Public Finance,Technology and Data Science,"Trade, Growth, and Regional Economics"
0,facilitating learning and discoveryoriented in...,0,0,0,0,0,0,0,0,1,0,0,1
1,responding adaptively to the covidcrisis in in...,0,0,0,0,0,1,0,0,1,0,0,0
4,page of banerjee et al bmc public health intro...,0,0,0,0,0,1,0,0,0,0,0,0
5,antibiotics due to their higher amr potential ...,0,0,0,0,0,1,0,0,0,0,0,0
6,wuniversity of california san diego la jolla c...,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,journal of the european economic association s...,0,0,1,0,0,0,1,0,1,0,0,0
241,may aea papers and proceedings refuses to have...,0,0,0,0,1,0,1,0,0,0,0,0
242,how edtech can help leapfrog progress in educa...,0,0,1,0,0,0,0,0,0,0,1,0
243,types of adhd faraone et al used structured di...,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
df.to_excel("train_final2.xlsx", index=False)